***Welcome to Medical image processing in Python***<br/>

Presented by: Reza Saadatyar (2024-2025) <br/>
E-mail: Reza.Saadatyar@outlook.com 

**Import the require library**

In [1]:
import sys
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from colorama import Back, Fore

# from skimage.viewer import ImageViewer
# from OOP import Pre_Processing_R2g
# from Functions import image_resizer

**1. Set Image Path & Load data**

In [7]:
import os
from itertools import chain

class DirectoryReader:
    """
    A class for reading and processing files in a directory with a specific format.
    """
    def __init__(self, directory_path: str, format_type: str) -> None:
        """
        Initialize the DirectoryReader with a directory path and file format.

        :param directory_path: Path to the directory to scan for files.
        :param format_type: File format (extension) to filter files, e.g., ".tif".
        """        
        self.files: list[str] = [] # Stores the names of all files matching the specified format.
        self.full_path: list[str] = [] # Stores the full paths of all matching files.
        self.folder_path: list[str] = [] # Stores the unique folder paths containing the files.
        self.subfolder: list[list[str]] = [] # Stores the names of subfolders for each directory.
        self.format_type: str = format_type # Stores the file format to filter (e.g., ".tif").
        self.directory_path: str = directory_path # Stores the root directory path to scan.
        self._scan_directory()  # Perform the directory scanning process.

    def _scan_directory(self) -> None:
        for root, subfolder_name, files_name in os.walk(self.directory_path):  # Traverse the directory tree
            root = root.replace("\\", "/")  # Replace backslashes with forward slashes for cross-platform compatibility

            for file in files_name:
                if file.endswith(self.format_type):  # Check if the file ends with the specified format
                    self.files.append(file)  # Append the file name to the files list

                    if root not in self.folder_path:  # Check if the root folder is not already in the folder_paths list
                        self.folder_path.append(root)  # If not, append the root folder to the folder_paths list

                    self.full_path.append(os.path.join(root, file).replace("\\", "/"))  # Append the full file path

                    # Ensure subfolder names are unique and non-empty
                    if subfolder_name not in self.subfolder and subfolder_name != []:
                        self.subfolder.append(subfolder_name)  # Append subfolder names to subfolders list

    @property
    def all_file_paths(self) -> list[str]:
        """
        Retrieve all full file paths for files with the specified format.

        :return: List of full file paths.
        """
        return self.full_path

    @property
    def filenames(self) -> list[str]:
        """
        Retrieve the list of filenames.

        :return: List of filenames.
        """
        return self.files

    @property
    def folder_paths(self) -> list[str]:
        """
        Retrieve the list of folder paths containing the files.

        :return: List of folder paths.
        """
        return self.folder_path

    @property
    def subfoldernames(self) -> list[str]:
        """
        Retrieve a flattened list of subfolder names.

        :return: Flattened list of subfolder names.
        """
        return list(chain.from_iterable(self.subfolder))

In [8]:
directory_path = "D:/Medical-Image-Processing/Data/Inputs"

# Create an instance of DirectoryReader with the directory path and file format
file_reader = DirectoryReader(directory_path, format_type="tif")

path__with_file = file_reader.all_file_paths  # Get the list of all file paths
folder_paths = file_reader.folder_paths  # Get the list of folder paths containing the files
files = file_reader.filenames  # Get the list of filenames
subfolders = file_reader.subfoldernames  # Get the flattened list of subfolder names

print(Fore.GREEN + f"{path__with_file = }""\n" + Fore.BLUE + f"{folder_paths = }""\n" + Fore.MAGENTA + f"{files = }"+
      "\n" + Fore.CYAN + f"{subfolders = }")

path__with_file = ['D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma49_042003_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma49_042003_benign2_ccd.tif', 'D:/Medical-Im

*Image-Width: n*<br/>
*Image-Height: m*<br/>
*Channels: c*<br/>
*Planes: p*<br/>
*Grayscale: (p, m, n)*<br/>
*RGB: (p, m, n, c)*<br/>

**2. Convert the image into an array**

In [15]:
import numpy as np  # Import numpy for array manipulation
from skimage import io  # Import scikit-image library for image I/O operations
from Functions import directory_reader

class ImageLoader:
    """
    A class to load and process images from a directory into a NumPy array.
    """

    def __init__(self, directory_path: str, format_type: str) -> None:
        """
        Initialize the ImageLoader class.

        :param directory_path: Root directory containing the images.
        :param format_type: File format (extension) of the images (e.g., ".tif").
        """
        self.images = None  # Placeholder for loaded image array
        self.img_width = None  # Width of the images
        self.img_height = None  # Height of the images
        self.img_channels = None  # Number of color channels in the images
        self.format_type: str = format_type  # File format to filter (e.g., ".tif")
        self.directory_path: str = directory_path  # Root directory path to scan

    @property
    def load_dataset(self) -> np.ndarray:
        """
        Load images from the specified directory into a NumPy array.

        :return: NumPy array containing the loaded images.
        """
        # Initialize the DirectoryReader to get file paths
        dir_reader = directory_reader.DirectoryReader(self.directory_path, self.format_type)
        all_path = dir_reader.all_file_paths

        if not all_path:
            raise ValueError("No files found in the specified directory.")

        num_files = len(all_path)  # Total number of image files
        first_image = io.imread(all_path[0])  # Load the first image to determine its dimensions

        # Check if the image is grayscale or colored
        if first_image.ndim == 2:  # Grayscale image
            self.img_height, self.img_width = first_image.shape  # Get image dimensions
            self.images = np.zeros(  # Initialize NumPy array for grayscale images
                (num_files, self.img_height, self.img_width),
                dtype=np.uint8  # Pixel values are stored as unsigned 8-bit integers
            )
        else:  # Colored image (e.g., RGB)
            self.img_height, self.img_width, self.img_channels = first_image.shape  # Get image dimensions
            self.images = np.zeros(  # Initialize NumPy array for colored images
                (num_files, self.img_height, self.img_width, self.img_channels),
                dtype=np.uint8
            )

        # Load all images into the NumPy array
        for idx, file_path in enumerate(all_path):
            self.images[idx] = io.imread(file_path)  # Read and store each image

        return self.images  # Return the loaded images as a NumPy array

**2.1. Convert the images in the "Inputs" folder to a NumPy array**

In [16]:
file_path = "D:/Medical-Image-Processing/Data/Inputs/"
inputs = ImageLoader(file_path, format_type="tif") # A class to load and process images from file paths into a NumPy array.
inputs_array = inputs.load_dataset # Load images from the provided file paths into a NumPy array.
print(Fore.GREEN + f"{inputs_array.shape = }") 

inputs_array.shape = (58, 768, 896, 3)


In [12]:
file_path = "D:/Medical-Image-Processing/Data/Masks/"
masks = ImageLoader(file_path, format_type="TIF") # A class to load and process images from file paths into a NumPy array.
masks_array = masks.load_dataset # Load images from the provided file paths into a NumPy array.
print(Fore.GREEN + f"{masks_array.shape = }") 

masks_array.shape = (58, 768, 896)


**2.2. Convert the images in the "Masks" folder to boolean**

In [8]:
data = io.imread(files_inputs[0])  # Load a file for obtaining size data
img_height = data.shape[0]         # Get the height of the image
img_width = data.shape[1]          # Get the width of the image
img_channels = data.shape[2]       # Get the number of channels in the image
labels = np.zeros((len(files_inputs), img_height, img_width, 2), dtype = bool)  # Shape: [num_files, H, W, 2]

sys.stdout.flush()
for ind, _ in tqdm(enumerate(files_inputs)):  # Progressively iterate through all the input files
    mask = np.squeeze(io.imread(files_masks[ind])).astype(bool)  # Load and convert each mask to boolean
    labels[ind, :, :, 0] = ~mask  # Background (inverse of mask)
    labels[ind, :, :, 1] = mask   # Foreground (actual mask)

58it [00:00, 175.09it/s]


**3. Image Processor**<br/>
***3.1. [Image Resizing](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.resize)***<br/>
- `Standardizing Image Dimensions:` Machine learning models, like CNNs, need input data with fixed dimensions. For instance, if a model requires images of size 224x224x3, all input images must be resized to that shape.<br/>
- `Reducing Computational Load:`Resizing images to smaller dimensions lowers computational costs, particularly with large datasets, and aids in faster training or inference for deep learning models.

**Augmentation, Re_Color, & Im_Saving**<br/>

In [13]:
import os
import numpy as np  # Import NumPy for array manipulation
from skimage import io, color, transform  # Import transform module for image resizing

# ================================= Class for resizing & converting images ====================================
class ImageProcessor:
    """
    A class for processing image data, including resizing.
    """

    # def __init__(self, img_data: np.ndarray):
    def __init__(self):
        """
        Initialize the ImageProcessor class.

        :param img_data: A numpy array of shape [num_images, height, width, channels].
        """

        self.img_gray = None
        self.file_names = None
        # self.img_data = img_data  # Store the input image data

    # ---------------------------------------------- Resizes ---------------------------------------------------
    def resize_images(self, img_height_resized: int, img_width_resized: int) -> np.ndarray:
        """
        Resizes a batch of images to the specified height, width, and channels.

        :param img_height_resized: Target height of the images after resizing.
        :param img_width_resized: Target width of the images after resizing.
        :return: A numpy array of resized images.
        """

        # Check if the input is grayscale (3D) or colored (4D)
        if self.img_data.ndim == 3:  # Grayscale images (no color channels)
            img_channels = 1  # Grayscale implies 1 channel
            # Initialize an array to store resized grayscale images
            resized_imgs = np.zeros(
                (self.img_data.shape[0], img_height_resized, img_width_resized),
                dtype=np.uint8
            )

            # Loop through each grayscale image in the batch
            for i in range(self.img_data.shape[0]):
                # Resize the image to the target dimensions and store it
                resized_imgs[i] = transform.resize(
                    self.img_data[i],
                    (img_height_resized, img_width_resized),
                    preserve_range=True  # Preserve the range of pixel values
                )

        else:  # Colored images (4D array with channels)
            img_channels = self.img_data.shape[-1]  # Get the number of color channels
            # Initialize an array to store resized colored images
            resized_imgs = np.zeros(
                (self.img_data.shape[0], img_height_resized, img_width_resized, img_channels),
                dtype=np.uint8
            )

            # Loop through each colored image in the batch
            for i in range(self.img_data.shape[0]):
                # Resize the image to the target dimensions and store it
                resized_imgs[i] = transform.resize(
                    self.img_data[i],
                    (img_height_resized, img_width_resized, img_channels),
                    preserve_range=True  # Preserve the range of pixel values
                )

        return resized_imgs  # Return the resized images
    
    # ---------------------------------------- RGB images to grayscale -----------------------------------------
    def RGB2Gray(self, files_path: str) -> np.ndarray:
        
        # Get the full path and filenames of files in the folder (excluding subfolders)
        files_with_paths = [os.path.join(files_path, f) for f in os.listdir(files_path) 
                    if os.path.isfile(os.path.join(files_path, f))]
        
        files_with_paths = [file for file in files_with_paths if os.path.splitext(file)[1].casefold() in 
                           ['.png', '.tif', '.jpg']]

        # Separate file names from the paths
        self.file_names = [os.path.basename(file) for file in files_with_paths]

        img_num = len(files_with_paths)
        img_height, img_width, _ = io.imread(files_with_paths[0]).shape
        self.img_gray = np.zeros((img_num, img_height, img_width), dtype=np.uint8)

        for i in range(img_num):
            self.img_gray[i] = (color.rgb2gray(io.imread(files_with_paths[i])) * 255).astype(np.uint8) # scale back to [0, 255]
            
        return self.img_gray
    
    def save_img_gray(self, path_save):
        os.makedirs(os.path.join(path_save, 'Gray image/'), exist_ok=True) # Ensure the temporary folder is created
        for ind, filename in enumerate(self.file_names):
            io.imsave(fname='{}{}'.format(path_save + 'Gray image/', filename), arr=self.img_gray[ind])
        
        print(Fore.GREEN + Back.BLACK + "The images have been saved successfully.")



**3.1. Image Resizing**

In [14]:
# Create an instance of the ImageProcessor class with the input image array
img = ImageProcessor(inputs_array)  # `inputs_array` is the original batch of images to process

# Call the `resize_images` method to resize the images to the target dimensions (255x255)
resized_images = img.resize_images(img_height_resized=255, img_width_resized=255)  # Resize all images to 255x255

# Print a message showing the original and resized image shapes for verification
print(Fore.GREEN + Back.BLACK + f"Resizing images from {inputs_array.shape} to {resized_images.shape}")

TypeError: ImageProcessor.__init__() takes 1 positional argument but 2 were given

In [10]:
files_path = "D:/Medical-Image-Processing/Data/Inputs/"
img = ImageProcessor()
output_gray = img.RGB2Gray(files_path)
print(Fore.GREEN + Back.BLACK + f"{output_gray.shape = }")

output_gray.shape = (51, 768, 896)


In [11]:
path_save = 'D:/Medical-Image-Processing/Data/'
img.save_img_gray(path_save)

The images have been saved successfully.


Resizing images from (58, 768, 896, 3) to (58, 255, 255, 3)


In [20]:
# Create an instance of the ImageProcessor class with the mask image array
img1 = ImageProcessor(masks_array)

# Call the `resize_images` method to resize the images to the target dimensions (255x255)
resized_images = img1.resize_images(img_height_resized=255, img_width_resized=255)
print(Fore.GREEN + Back.BLACK + f"Resizing images from {masks_array.shape} to {resized_images.shape}") 

Resizing images from (58, 768, 896) to (58, 255, 255)


In [ ]:
import os
import glob
import shutil

from skimage import transform, color  
from tensorflow import keras

#  ================================= Class for resizing & converting images ====================================
class Image:
    def __init__(self, img_data: np.ndarray):
        """
        :param imgs: A numpy array of shape [num_images, height, width, channels].
        """
        self.img_data = img_data
        
    
   

# ================================== Class for augmentation (rotation) =========================================
class Augmentation:
    def __init__(self, num_augmented_imag: int, imag_files_path: str, imag_augmented_path: str):
        """
        Applies image augmentation (rotation) to images in the specified directory and saves them.

        :param num_augmented_images: Number of augmented images to generate.
        :param imag_files_path: Path to the directory containing the images.
        :param imag_augmented_path: Path to the directory to save augmented images.
        """
        
        self.imag_files_path = imag_files_path
        self.num_augmented_imag = num_augmented_imag
        self.imag_augmented_path = imag_augmented_path
        
    def augmented_images(self) -> None:
        
        dat = io.imread(glob.glob(self.imag_files_path + '/*')[0])
        self.imag_augmented_path = os.path.join(self.imag_augmented_path, 'Rotated/')
        os.makedirs(self.imag_augmented_path, exist_ok=True)  # Ensure the temporary folder is created
        # Create a temporary folder inside the original folder for processing
        TEMP_DIR = os.path.join(self.imag_files_path, 'Temp/')
        os.makedirs(TEMP_DIR, exist_ok=True)  # Ensure the temporary folder is created
        
        # Copy all files from the main folder to the temporary folder
        for filename in os.listdir(self.imag_files_path):
            if filename.casefold().endswith(('.tif', '.jpg', '.png')):  # Correct usage with a tuple
                shutil.copy(os.path.join(self.imag_files_path, filename), os.path.join(TEMP_DIR, filename))

        # Set up the ImageDataGenerator for image augmentation
        Data_Gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=30)  # Rotate images randomly up to 30 degrees
        # Use flow_from_directory to process the images in the Temp folder
        img_aug = Data_Gen.flow_from_directory(
            self.imag_files_path,     # Parent directory of Temp
            classes=['Temp'],         # Specify the subfolder 'Temp' as the target
            batch_size=1,             # Process one image at a time
            save_to_dir=self.imag_augmented_path,  # Save augmented images to the Rotated folder
            save_prefix='Aug',        # Prefix for augmented images
            target_size=(dat.shape[0], dat.shape[1]),  # Resize images to the specified dimensions
            class_mode=None           # No labels, as we're working with unclassified images
        )
        
        for _ in range(self.num_augmented_imag):  # Generate augmented images and save them
            next(img_aug)  # Process the next image and save it

        shutil.rmtree(TEMP_DIR)  # Delete the temporary folder and its contents after processing

    # ---------------------------------------------- Plot ------------------------------------------------------
    def plot_img_original_augment(self, num_img: int) -> None:
        
        _, axs = plt.subplots(nrows=2, ncols=num_img)
        
        # Check if num_img is 1 (special case for 1 image)
        if num_img == 1:
            # Display images on the first row
            # io.imread(files_inputs[0])[:, :, :3].shape
            axs[0].imshow(io.imread(glob.glob(self.imag_files_path + '/*')[0]), cmap='gray')
            axs[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
            [spine.set_visible(False) for spine in axs[0].spines.values()]  # Hide all spines
            axs[0].set_ylabel("Original Images", fontsize=12, labelpad=10)  # Y-axis label for the first row
            
            # Display images on the second row
            axs[1].imshow(io.imread(glob.glob(self.imag_augmented_path + '/*')[0]), cmap='gray')
            axs[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
            [spine.set_visible(False) for spine in axs[1].spines.values()]  # Hide all spines
            axs[1].set_ylabel("Augmented Images", fontsize=12, labelpad=10)  # Y-axis label for the second row

        else:
            for i in range(num_img):
                # Display images on the first row
                axs[0, i].imshow(io.imread(glob.glob(self.imag_files_path + '/*')[i]), cmap='gray')
                axs[0, i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
                [spine.set_visible(False) for spine in axs[0, i].spines.values()]  # Hide all spines

                # Display images on the second row
                axs[1, i].imshow(io.imread(glob.glob(self.imag_augmented_path + '/*')[i]), cmap='gray')
                axs[1, i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
                [spine.set_visible(False) for spine in axs[1, i].spines.values()]  # Hide all spines

            # Add ylabel for each row (only set ylabel for the first column of each row)
            axs[0, 0].set_ylabel("Original Images", fontsize=12, labelpad=10)  # Y-axis label for the first row
            axs[1, 0].set_ylabel("Augmented Images", fontsize=12, labelpad=10)  # Y-axis label for the second row

        # Adjust layout to make sure images and titles don't overlap
        plt.tight_layout()

        # Auto-scale to fit the images in the figure area
        plt.autoscale(enable=True, axis='both', tight=True)
        plt.show()



In [95]:
import os

# Define the folder path
files_path = "D:/Medical-Image-Processing/Data/Inputs"

# Get the full path and filenames of files in the folder (excluding subfolders)
files_with_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) 
                    if os.path.isfile(os.path.join(folder_path, f))]

print(files_with_paths)

['D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma49_042003_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma49_042003_benign2_ccd.tif', 'D:/Medical-Image-P

1 --> ytma12_010804_benign2_ccd.tif
2 --> ytma12_010804_benign3_ccd.tif
3 --> ytma12_010804_malignant1_ccd.tif
4 --> ytma12_010804_malignant2_ccd.tif
5 --> ytma12_010804_malignant3_ccd.tif
6 --> ytma23_022103_benign1_ccd.tif
7 --> ytma23_022103_benign2_ccd.tif
8 --> ytma23_022103_benign3_ccd.tif
9 --> ytma23_022103_malignant1_ccd.tif
10 --> ytma23_022103_malignant2_ccd.tif
11 --> ytma23_022103_malignant3_ccd.tif
12 --> ytma49_042003_benign1_ccd.tif
13 --> ytma49_042003_benign2_ccd.tif
14 --> ytma49_042003_benign3_ccd.tif
15 --> ytma49_042003_malignant1_ccd.tif
16 --> ytma49_042003_malignant2_ccd.tif
17 --> ytma49_042003_malignant3_ccd.tif
18 --> ytma49_042203_benign1_ccd.tif
19 --> ytma49_042203_benign2_ccd.tif
20 --> ytma49_042203_benign3_ccd.tif
21 --> ytma49_042203_malignant1_ccd.tif
22 --> ytma49_042203_malignant2_ccd.tif
23 --> ytma49_042203_malignant3_ccd.tif
24 --> ytma49_042403_benign1_ccd.tif
25 --> ytma49_042403_benign2_ccd.tif
26 --> ytma49_042403_benign3_ccd.tif
27 --> ytma

In [ ]:

New = 'D:/Python/Breast/'

def Pre_Process_Im_Saving(Path_Images, Path_Output, Tensor):
    
    for i, filename in enumerate(os.listdir(Path_Images)):
        
        imsave(fname='{}{}'.format(Path_Output, filename),
               arr=Tensor[i])
        
        print('{}: {}'.format(i, filename))
    
Pre_Process_Im_Saving(IMAGE_PATH, New, Gray_Scale)

In [63]:
a = img.rgb2gray_scale()
a.shape

(58, 768, 896)

In [60]:
num_augmented_imag = 3
# Path to the folder containing the original images
imag_files_path = 'D:/Medical-Image-Processing/Data/Inputs/A/B/C/'

# Path where augmented images will be saved
imag_augmented_path = 'D:/Medical-Image-Processing/Data/'
augm = Augmentation(num_augmented_imag, imag_files_path, imag_augmented_path)
augm.augmented_images()
# resizer.augmented_images(num_augmented_imag, imag_files_path, imag_augmented_path)

Found 2 images belonging to 1 classes.


In [ ]:
augm.plot_img_original_augment(num_img=2)

In [20]:
dat = io.imread(glob.glob(imag_files_path + '/*')[0])
dat.shape[0]

768